# 第10回 アルゴリズム入門：問題の複雑さ

___
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tsuboshun/begin-python/blob/gh-pages/_sources/workbook/lecture10.ipynb)

___

## この授業で学ぶこと

前回の授業で挿入ソートとクイックソートを紹介し、それらの実行速度に差があることを見た。特にデータ数を増やしたときの実行時間の増え方に大きな違いがあることを見た。今回は、アルゴリズムの実行時間を見積もるための考え方である計算量について学ぶ。また、それをもとにP・NPという問題の難しさを表す分類について学ぶ。

### 準備

In [ ]:
import time
import random
import itertools
import matplotlib.pyplot as plt

In [ ]:
N = 10**6
nums = []
for i in range(N):
    nums.append(random.random())

## 計算量

**計算量**とは、データ数を $N$ と書くとき、計算ステップ数が $N$ に対してどのように増えるかを表す指標である。

例として、次の2つのプログラムの計算時間を考える。

In [ ]:
def simple_sum(N):
    count = 0
    for i in range(1, N+1):
        count += 1
    return count

In [ ]:
def simple_sum2(N):
    count = 0
    for i in range(1, N+1):
        for j in range(1, N+1):
            count += 1
    return count

`simple_sum()` は一重ループ、`simple_sum2()` は二重ループで、単純な足し算を実行する関数である。各ループ回数を引数 `N` で指定する。
これらの関数の実行時間を次の `exec_time()` で測定する。

In [ ]:
def exec_time(func, N):
    start = time.time()
    func(N)
    end = time.time()
    return end - start

`exec_time()` について補足する。前回とは違い、今回は引数として関数を渡している！
第4回の授業で、「Pythonに出てくるほとんど全ての要素はオブジェクトとして作られている」という意味ありげな説明をしたが、実は関数もオブジェクトである。
したがって、データと同様に引数に渡すということも可能になっている。

さて、データ数に対するそれぞれの関数の実行時間は次のようになる。

In [ ]:
n = []
t1 = []
t2 = []
for i in range(1,10):
    size = i * 2 * 10**3
    n.append(size)
    t1.append(exec_time(simple_sum, size))
    t2.append(exec_time(simple_sum2, size))
    
# プロットの入れ物の用意
fig, ax = plt.subplots(figsize=(8,6))

# プロット
ax.plot(n, t1, label="simple_sum()")
ax.plot(n, t2, label="simple_sum2()")

# 軸ラベルなどの設定
ax.set_xlabel('size', size=14)
ax.set_ylabel('time[s]', size=14)
ax.tick_params(labelsize=14)

# 表示
plt.legend(fontsize=14)
plt.show()

## 巡回セールスマン問題

In [ ]:
def generate_distances(N):
    ret = [[0] * N for i in range(N)]  # N * N の 2次元リストの用意
    for i in range(N):
        for j in range(N):
            if i < j:
                d = random.randint(1, 10)  # 1以上10以下の整数をランダムに生成
                ret[i][j] = d
                ret[j][i] = d
    return ret


def solve_tsp(distances):
    # 都市を回る順番について、全ての組み合わせを生成する
    city_combinations = itertools.permutations(range(len(distances)))

    # 最短の経路と距離を初期化する
    shortest_distance = 10**8  # 十分に大きい値であれば何でも良い
    shortest_path = None

    # 全ての経路を試す
    for path in city_combinations:
        total_distance = 0
        for i in range(len(path)-1):
            total_distance += distances[path[i]][path[i+1]]
        # 最短経路を更新する
        if total_distance < shortest_distance:
            shortest_distance = total_distance
            shortest_path = path

    # 結果を出力する
    print("最短距離:", shortest_distance)
    print("最短経路:", shortest_path)

In [ ]:
N = 8

# 都市間の距離を表す2次元リスト
distances = generate_distances(N)

# 最短経路の探索
start = time.time()
solve_tsp(distances)
end = time.time()

print(f"実行時間: {end - start:.4f}秒")